In [33]:
import pandas as pd
import pypsa
import yaml
import glob

In [34]:
### config to be put in config.yaml
with open('../config.yaml', 'r') as configfile:
    config = yaml.safe_load(configfile)
    
TY = config['scenario']['target_years'][0]

simulation_year = config['scenario']['simulation_years'][0]

short_names = config['carriers_short_names']

basedir = config['io']['databasedir']+f'scenario_{TY}_{simulation_year}/'

scenario = f'National estimates {TY}' # snakemake input!

network_f = f'../networks/pilot_elec-vre_TY{TY}_{simulation_year}.nc'

In [35]:
network = pypsa.Network(network_f)
network.name = 'DestinE Pilot with VRE profiles and hydro'

Index(['UA00 nuclear', 'UA00 coal', 'UA00 ror', 'UA00 onwind', 'UA00 CSP',
       'UA00 solar', 'UA00 other RE'],
      dtype='object', name='name')
Index(['UA01 - Demand', 'UA00 - Demand'], dtype='object', name='name')
INFO:pypsa.io:Imported network pilot_elec-vre_TY2030_2010.nc has buses, carriers, generators, links, loads, stores


In [84]:
### add natural inflows
inflow = pd.read_csv(basedir+f'hydro_inflows_{simulation_year}.csv', header=0, index_col=0)

hydro_gens = pd.DataFrame(index=inflow.columns)
hydro_gens['carrier'] = [c.split(' - ')[1] for c in inflow.columns]
hydro_gens['carrier_short'] = hydro_gens['carrier'].map(short_names)
hydro_gens['node'] = [c.split(' - ')[0] for c in inflow.columns]
hydro_gens['name_short'] = hydro_gens[['node', 'carrier_short']].agg(' '.join, axis=1)

inflow.columns = inflow.columns.map(hydro_gens['name_short'])
inflow.index = network.snapshots

inflow = inflow.loc[:, inflow.columns[inflow.columns.isin(network.stores.index)]]
buses = inflow.columns
inflow.columns = [c+' inflow' for c in inflow.columns]

network.add('Carrier', 'surface_runoff')
network.madd(
    'Generator',
    inflow.columns,
    bus = buses,
    carrier = 'surface_runoff',
    p_nom = inflow.max(),
    p_max_pu = inflow / inflow.max(),
)

Index(['AL00 hydro inflow', 'AT00 pondage inflow', 'AT00 hydro inflow',
       'AT00 OLPHS inflow', 'BA00 pondage inflow', 'BA00 hydro inflow',
       'BA00 OLPHS inflow', 'BG00 hydro inflow', 'BG00 OLPHS inflow',
       'CH00 hydro inflow', 'CH00 OLPHS inflow', 'CZ00 hydro inflow',
       'CZ00 OLPHS inflow', 'DE00 hydro inflow', 'DE00 OLPHS inflow',
       'ES00 hydro inflow', 'ES00 OLPHS inflow', 'FI00 hydro inflow',
       'FR00 hydro inflow', 'GR00 pondage inflow', 'GR00 hydro inflow',
       'GR00 OLPHS inflow', 'HR00 pondage inflow', 'HR00 hydro inflow',
       'HR00 OLPHS inflow', 'ITCA hydro inflow', 'ITCN hydro inflow',
       'ITCS hydro inflow', 'ITCS OLPHS inflow', 'ITN1 hydro inflow',
       'ITN1 OLPHS inflow', 'ITS1 hydro inflow', 'ITSA hydro inflow',
       'ITSI hydro inflow', 'ITSI OLPHS inflow', 'LV00 pondage inflow',
       'ME00 hydro inflow', 'MK00 hydro inflow', 'NOM1 OLPHS inflow',
       'NON1 OLPHS inflow', 'NOS0 OLPHS inflow', 'PL00 hydro inflow',
       'PL

In [139]:
hydro_files = sorted(glob.glob(basedir+f'hydro_uniform_*_{simulation_year}.csv'))

for hydro_f in hydro_files:
    limits = pd.read_csv(hydro_f, index_col=0, header=0)

    hydro_sus = pd.DataFrame(index=limits.columns)
    hydro_sus['carrier'] = [c.split(' - ')[1] for c in limits.columns]
    hydro_sus['carrier_short'] = hydro_sus['carrier'].map(short_names)
    hydro_sus['node'] = [c.split(' - ')[0] for c in limits.columns]
    hydro_sus['name_short'] = hydro_sus[['node', 'carrier_short']].agg(' '.join, axis=1)

    limits.columns = limits.columns.map(hydro_sus['name_short'])
    limits.index = network.snapshots
    limits.drop(limits.columns[~limits.columns.isin(network.stores.index)], axis=1, inplace=True)

    constraint = hydro_f.split('_')[-2]

    if constraint == 'Maximum Generated energy GWh per week':
        ### Limits have been equally distributed to hours within week
        ### set p_max_pu of dispatching link

        disp_links = network.links.index[network.links.bus0.isin(hydro_sus.name_short)]
        max_disp_pu = limits / network.links.set_index(network.links.bus0).p_nom[limits.columns]
        max_disp_pu.columns = [c+' dispatch' for c in max_disp_pu.columns]
        network.links_t.p_max_pu[disp_links] = max_disp_pu.clip(upper=1.)

    elif constraint == 'Maximum Generation MW':
        ### set p_max_pu of dispatching link

        disp_links = network.links.index[network.links.bus0.isin(hydro_sus.name_short)]
        max_disp_pu = limits / network.links.set_index(network.links.bus0).p_nom[limits.columns]
        max_disp_pu.columns = [c+' dispatch' for c in max_disp_pu.columns]
        network.links_t.p_max_pu[disp_links] = max_disp_pu.clip(upper=1.)

    elif constraint == 'Maximum Pumped Energy GWh per week':
        ### Limits have been equally distributed to hours within week
        ### set p_max_pu of storing link

        store_links = network.links.index[network.links.bus1.isin(hydro_sus.name_short)]
        max_store_pu = limits / network.links.set_index(network.links.bus1).p_nom[limits.columns]
        max_store_pu.columns = [c+' store' for c in max_store_pu.columns]
        network.links_t.p_max_pu[store_links] = max_store_pu.clip(upper=1.)

    elif constraint == 'Maximum Pumping MW':
        ### set p_max_pu of storing link

        store_links = network.links.index[network.links.bus1.isin(hydro_sus.name_short)]
        max_store_pu = limits / network.links.set_index(network.links.bus1).p_nom[limits.columns]
        max_store_pu.columns = [c+' store' for c in max_store_pu.columns]
        network.links_t.p_max_pu[store_links] = max_store_pu.clip(upper=1.)

    elif constraint == 'Maximum Reservoir level, historical (ratio) ':
        ### set e_max_pu of store
        network.stores_t.e_max_pu[limits.columns] = limits

    elif constraint == 'Minimum Generated energy GWh per week':
        ### Limits have been equally distributed to hours within week
        ### set p_min_pu of dispatching link

        disp_links = network.links.index[network.links.bus0.isin(hydro_sus.name_short)]
        min_disp_pu = limits / network.links.set_index(network.links.bus0).p_nom[limits.columns]
        min_disp_pu.columns = [c+' dispatch' for c in min_disp_pu.columns]
        network.links_t.p_min_pu[disp_links] = min_disp_pu.clip(upper=1.)

    elif constraint == 'Minimum Generation MW':
        ### set p_min_pu of dispatching link

        disp_links = network.links.index[network.links.bus0.isin(hydro_sus.name_short)]
        min_disp_pu = limits / network.links.set_index(network.links.bus0).p_nom[limits.columns]
        min_disp_pu.columns = [c+' dispatch' for c in min_disp_pu.columns]
        network.links_t.p_min_pu[disp_links] = min_disp_pu.clip(upper=1.)

    elif constraint == 'Minimum Pumped energy GWh per week':
        ### Limits have been equally distributed to hours within week
        ### set p_min_pu of storing link

        store_links = network.links.index[network.links.bus1.isin(hydro_sus.name_short)]
        min_store_pu = limits / network.links.set_index(network.links.bus1).p_nom[limits.columns]
        min_store_pu.columns = [c+' store' for c in min_store_pu.columns]
        network.links_t.p_min_pu[store_links] = min_store_pu.clip(upper=1.)

    elif constraint == 'Minimum Pumping MW':
        ### set p_min_pu of storing link

        store_links = network.links.index[network.links.bus1.isin(hydro_sus.name_short)]
        min_store_pu = limits / network.links.set_index(network.links.bus1).p_nom[limits.columns]
        min_store_pu.columns = [c+' store' for c in min_store_pu.columns]
        network.links_t.p_min_pu[store_links] = min_store_pu.clip(upper=1.)

    elif constraint == 'Minimum Reservoir level, historical (ratio) ':
        ### set e_min_pu of store
        network.stores_t.e_min_pu[limits.columns] = limits

    elif constraint == 'Minimum Reservoir level, technical (ratio) ':
        ### set e_min_pu of store
        network.stores_t.e_min_pu[limits.columns] = limits

    elif constraint == 'Reservoir level at beginning of week (ratio) ':
        ### e_max_pu = e_min_pu at beginning of week
        e_min_pu = limits.fillna(0.)
        e_max_pu = limits.fillna(1.)
        network.stores_t.e_min_pu[e_min_pu.columns] = e_min_pu
        network.stores_t.e_max_pu[e_max_pu.columns] = e_max_pu

    else:
        print(f'Warning hydro constraint not specified for constraint = {constraint}')